In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

featuresdf=pd.read_csv('./csv/set4.csv', index_col=0)
featuresdf = featuresdf.reset_index()
set4fake = featuresdf.copy()
for column in ['acceleration_x','acceleration_y','acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z','second', 'Speed']:
    justnullsdf = set4fake[set4fake[column].isnull()]
    bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
    rows = 0
    for i in range (len(bookingids)):
        bookingidx = set4fake[set4fake['bookingID']==bookingids[i]]
        if bookingidx[column].isnull().all().all():
            set4fake = set4fake.drop(set4fake[set4fake.bookingID == bookingids[i]].index)
            
uniqueValues = set4fake['bookingID'].unique()
uniqueValues = np.sort(uniqueValues)
bookingids=featuresdf['bookingID'].unique()


In [ ]:
print(set4fake.shape)
set4_1fake = set4fake[set4fake['bookingID'].isin(uniqueValues[0:1650])]
print(set4_1fake.shape)
set4_2fake = set4fake[set4fake['bookingID'].isin(uniqueValues[1650:2900])]
print(set4_2fake.shape)
set4_3fake = set4fake[set4fake['bookingID'].isin(uniqueValues[2900:])]
print(set4_3fake.shape)
print(set4_1fake.shape[0] + set4_2fake.shape[0] + set4_3fake.shape[0])

(188106, 12)
(66258, 12)
(50640, 12)
(71208, 12)
188106


# SET 4-2

In [ ]:
# THIS IS ONLY FOR SET 3_2fake acceleration_x
justnullsdf = set4_2fake[set4_2fake['acceleration_x'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_2fake[set4_2fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against acceleration_x
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_2fake = set4_2fake.drop(set4_2fake[set4_2fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['acceleration_x']) # finding the standard deviation of acceleration_x
    nulldf = bookingidx[bookingidx['acceleration_x'].isnull()] # make dataframe of records with acceleration_x being a null value
    for ind in nulldf.index: #retrieve the index of the row with acceleration_x = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 4] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_x'].empty or bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_x'].isnull().bool(): # if the next and next next record is null
                    estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1+1]['acceleration_x']) - std -std -std
            elif bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_x']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_2fake[set4_2fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'acceleration_x'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_2fake.at[int(realindex),'acceleration_x'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_2fake[set4_2fake.index.values == int(realindex)]['acceleration_x'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['acceleration_x'] # just for printing to debug
        print(f'the value back in set4_2fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 704374636692.0, index: 33, id: 5172373
the previous value is -0.52098787 and the after value is 0.4405331
estimated value is -0.040227385000000004
the value back in set4_2fake is now 129907   -0.040227
Name: acceleration_x, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_x, dtype: float64)
-----------------------
bookingid: 704374636692.0, index: 44, id: 1471
the previous value is 0.6328929 and the after value is 0.97444963
estimated value is 0.803671265
the value back in set4_2fake is now 36    0.803671
Name: acceleration_x, dtype: float64
the value back in bookingidx is now 36   -1.659919
Name: acceleration_x, dtype: float64
-----------------------
bookingid: 704374636692.0, index: 47, id: 243325
the previous value is 1.4001867 and the after value is -0.19431342
estimated value is 0.6029366399999999
the value back in set4_2fake is now 6019    0.602937
Name: acceleration_x, dtype: float64
the value back in bookingidx is now Series([], Name: 

In [ ]:
set4_2fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy           851
Bearing           1118
acceleration_x       0
acceleration_y    1215
acceleration_z     436
gyro_x             878
gyro_y             837
gyro_z            1021
second             662
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_2fake acceleration_y
justnullsdf = set4_2fake[set4_2fake['acceleration_y'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_2fake[set4_2fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against acceleration_y
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_2fake = set4_2fake.drop(set4_2fake[set4_2fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['acceleration_y']) # finding the standard deviation of acceleration_y
    nulldf = bookingidx[bookingidx['acceleration_y'].isnull()] # make dataframe of records with acceleration_y being a null value
    for ind in nulldf.index: #retrieve the index of the row with acceleration_y = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 5] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_y']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_2fake[set4_2fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'acceleration_y'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_2fake.at[int(realindex),'acceleration_y'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_2fake[set4_2fake.index.values == int(realindex)]['acceleration_y'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['acceleration_y'] # just for printing to debug
        print(f'the value back in set4_2fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 661424963633.0, index: 21, id: 1425
the previous value is -8.818893432617188 and the after value is -10.397097778320315
estimated value is -9.607995605468751
the value back in set4_2fake is now 35   -9.607996
Name: acceleration_y, dtype: float64
the value back in bookingidx is now 35   -9.930396
Name: acceleration_y, dtype: float64
-----------------------
bookingid: 584115552308.0, index: 24, id: 4763696
the previous value is -10.275823974609375 and the after value is -9.3513916015625
estimated value is -9.813607788085939
the value back in set4_2fake is now 119581   -9.813608
Name: acceleration_y, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_y, dtype: float64)
-----------------------
bookingid: 584115552308.0, index: 32, id: 4056
the previous value is -12.087454223632813 and the after value is -11.130422973632813
estimated value is -11.608938598632813
the value back in set4_2fake is now 100   -11.608939
Name: acceleration_y, dtype: float64

In [ ]:
set4_2fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy           850
Bearing           1117
acceleration_x       0
acceleration_y       0
acceleration_z     435
gyro_x             878
gyro_y             836
gyro_z            1020
second             662
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_2fake acceleration_z
justnullsdf = set4_2fake[set4_2fake['acceleration_z'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_2fake[set4_2fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against acceleration_z
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_2fake = set4_2fake.drop(set4_2fake[set4_2fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['acceleration_z']) # finding the standard deviation of acceleration_z
    nulldf = bookingidx[bookingidx['acceleration_z'].isnull()] # make dataframe of records with acceleration_z being a null value
    for ind in nulldf.index: #retrieve the index of the row with acceleration_z = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 6] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_z']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_2fake[set4_2fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'acceleration_z'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_2fake.at[int(realindex),'acceleration_z'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_2fake[set4_2fake.index.values == int(realindex)]['acceleration_z'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['acceleration_z'] # just for printing to debug
        print(f'the value back in set4_2fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 670014898253.0, index: 5, id: 11952
the previous value is -2.756923200000001 and the after value is -2.823961
estimated value is -2.790442100000001
the value back in set4_2fake is now 308   -2.790442
Name: acceleration_z, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_z, dtype: float64)
-----------------------
bookingid: 635655159934.0, index: 90, id: 5950591
the previous value is -2.9096740722656254 and the after value is -4.237405395507813
estimated value is -3.573539733886719
the value back in set4_2fake is now 149783   -3.57354
Name: acceleration_z, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_z, dtype: float64)
-----------------------
bookingid: 635655159934.0, index: 153, id: 3792314
the previous value is -3.874929809570313 and the after value is -4.9203369140625
estimated value is -4.397633361816407
the value back in set4_2fake is now 95376   -4.397633
Name: acceleration_z, dtype: float64
the value 

In [ ]:
set4_2fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy           850
Bearing           1116
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x             877
gyro_y             836
gyro_z            1020
second             662
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_2fake gyro_x
justnullsdf = set4_2fake[set4_2fake['gyro_x'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_2fake[set4_2fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against gyro_x
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_2fake = set4_2fake.drop(set4_2fake[set4_2fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['gyro_x']) # finding the standard deviation of gyro_x
    nulldf = bookingidx[bookingidx['gyro_x'].isnull()] # make dataframe of records with gyro_x being a null value
    for ind in nulldf.index: #retrieve the index of the row with gyro_x = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 7] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['gyro_x']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_2fake[set4_2fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'gyro_x'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_2fake.at[int(realindex),'gyro_x'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_2fake[set4_2fake.index.values == int(realindex)]['gyro_x'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['gyro_x'] # just for printing to debug
        print(f'the value back in set4_2fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 584115552390.0, index: 26, id: 4978695
the previous value is 0.05162684 and the after value is -0.012601277
estimated value is 0.0195127815
the value back in set4_2fake is now 125131    0.019513
Name: gyro_x, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_x, dtype: float64)
-----------------------
bookingid: 584115552390.0, index: 145, id: 41903
the previous value is -0.0235793979999999 and the after value is -0.08494517
estimated value is -0.05426228399999995
the value back in set4_2fake is now 1012   -0.054262
Name: gyro_x, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_x, dtype: float64)
-----------------------
bookingid: 584115552390.0, index: 148, id: 6211327
the previous value is 0.0898321 and the after value is 0.0808262
estimated value is 0.08532914999999999
the value back in set4_2fake is now 156531    0.085329
Name: gyro_x, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_x, dtype: float64)

In [ ]:
set4_2fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy           850
Bearing           1115
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x               0
gyro_y             836
gyro_z            1019
second             662
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_2fake gyro_y
justnullsdf = set4_2fake[set4_2fake['gyro_y'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_2fake[set4_2fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against gyro_y
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_2fake = set4_2fake.drop(set4_2fake[set4_2fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['gyro_y']) # finding the standard deviation of gyro_y
    nulldf = bookingidx[bookingidx['gyro_y'].isnull()] # make dataframe of records with gyro_y being a null value
    for ind in nulldf.index: #retrieve the index of the row with gyro_y = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 8] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['gyro_y']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_2fake[set4_2fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'gyro_y'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_2fake.at[int(realindex),'gyro_y'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_2fake[set4_2fake.index.values == int(realindex)]['gyro_y'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['gyro_y'] # just for printing to debug
        print(f'the value back in set4_2fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 730144440472.0, index: 13, id: 6118
the previous value is -0.0828011403999806 and the after value is -0.5220392284639485
estimated value is -0.3024201844319645
the value back in set4_2fake is now 154   -0.30242
Name: gyro_y, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_y, dtype: float64)
-----------------------
bookingid: 695784702012.0, index: 48, id: 11375
the previous value is -0.084741056 and the after value is -0.018767612
estimated value is -0.051754334
the value back in set4_2fake is now 296   -0.051754
Name: gyro_y, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_y, dtype: float64)
-----------------------
bookingid: 695784702012.0, index: 53, id: 3187493
the previous value is 0.06492093 and the after value is -0.14155152
estimated value is -0.03831529499999999
the value back in set4_2fake is now 80613   -0.038315
Name: gyro_y, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_y, dtype: float6

In [ ]:
set4_2fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy           849
Bearing           1115
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x               0
gyro_y               0
gyro_z            1019
second             661
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_2fake gyro_z
justnullsdf = set4_2fake[set4_2fake['gyro_z'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_2fake[set4_2fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against gyro_z
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_2fake = set4_2fake.drop(set4_2fake[set4_2fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['gyro_z']) # finding the standard deviation of gyro_z
    nulldf = bookingidx[bookingidx['gyro_z'].isnull()] # make dataframe of records with gyro_z being a null value
    for ind in nulldf.index: #retrieve the index of the row with gyro_z = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 9] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['gyro_z']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_2fake[set4_2fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'gyro_z'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_2fake.at[int(realindex),'gyro_z'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_2fake[set4_2fake.index.values == int(realindex)]['gyro_z'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['gyro_z'] # just for printing to debug
        print(f'the value back in set4_2fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 970662608898.0, index: 12, id: 5450
the previous value is 0.057858713 and the after value is 0.08473679
estimated value is 0.0712977515
the value back in set4_2fake is now 136    0.071298
Name: gyro_z, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_z, dtype: float64)
-----------------------
bookingid: 970662608898.0, index: 37, id: 542038
the previous value is 0.030369775 and the after value is -0.0227755
estimated value is 0.0037971375000000005
the value back in set4_2fake is now 13636    0.003797
Name: gyro_z, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_z, dtype: float64)
-----------------------
bookingid: 738734374954.0, index: 80, id: 9434
the previous value is -0.02687807 and the after value is -0.014660766
estimated value is -0.020769418
the value back in set4_2fake is now 250   -0.020769
Name: gyro_z, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_z, dtype: float64)
----------------------

In [ ]:
set4_2fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy           847
Bearing           1114
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x               0
gyro_y               0
gyro_z               0
second             661
Speed                0
dtype: int64

In [ ]:
set4_2fake.to_csv('set4_2done.csv')